In [ ]:
!gdown 1IZr7DYItUyP9uRHAMSILKTQrmBBcgCJQ

Downloading...
From: https://drive.google.com/uc?id=1IZr7DYItUyP9uRHAMSILKTQrmBBcgCJQ
To: /content/Animals_train.zip
100% 1.16G/1.16G [00:04<00:00, 273MB/s]


In [ ]:
from torchvision import datasets, transforms
from torchsummary import summary
from torch.utils import data
from torch.optim import Adam
from torch import nn
import torch.nn.functional as f
import torch
import tqdm
from datetime import datetime

In [ ]:
import zipfile
zip_path = 'Animals_train.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    # transforms.CenterCrop(224),
    # transforms.RandomCrop(200),
])

In [ ]:
dataset = datasets.ImageFolder(root='Data/Animals/train', transform=transform)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size
trainset, valset, testset = data.random_split(dataset, [train_size, val_size, test_size])
# train_dataset, val_dataset, test_dataset = data.random_split(dataset, [train_size, val_size, test_size])

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomCrop(size=200),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(size=200),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)


train_subset = MyDataset(trainset, transform=train_transforms)
val_subset = MyDataset(valset, transform=test_transforms)
test_subset = MyDataset(testset, transform=test_transforms)

In [ ]:
batch_size = 32
train_loader = data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(val_subset , batch_size=batch_size)
test_loader = data.DataLoader(test_subset, batch_size=batch_size)

In [ ]:
from torchvision import models


class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.convnext_large = models.convnext_base(pretrained=True)
    in_features = self.convnext_large.classifier[-1].in_features
    self.convnext_large.classifier[-1] = nn.Linear(in_features, 64)
    self.linear1 = nn.Linear(64, 8)

  def forward(self, x):
    x = self.convnext_base(x)
    x = f.sigmoid(x)
    x = self.linear1(x)
    return x


In [ ]:
model = MyModel()

In [ ]:
for param in model.convnext_large.parameters():
    param.requires_grad = False

for param in model.convnext_large.classifier[-1].parameters():
    param.requires_grad = True

# for param in model.convnext_base.classifier[1].parameters():
#     param.requires_grad = True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = model.to(device)
summary(model, (3, 220, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 55, 55]           6,272
       LayerNorm2d-2          [-1, 128, 55, 55]             256
            Conv2d-3          [-1, 128, 55, 55]           6,400
           Permute-4          [-1, 55, 55, 128]               0
         LayerNorm-5          [-1, 55, 55, 128]             256
            Linear-6          [-1, 55, 55, 512]          66,048
              GELU-7          [-1, 55, 55, 512]               0
            Linear-8          [-1, 55, 55, 128]          65,664
           Permute-9          [-1, 128, 55, 55]               0
  StochasticDepth-10          [-1, 128, 55, 55]               0
          CNBlock-11          [-1, 128, 55, 55]               0
           Conv2d-12          [-1, 128, 55, 55]           6,400
          Permute-13          [-1, 55, 55, 128]               0
        LayerNorm-14          [-1, 55, 

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[82, 123], gamma=0.1)

train_accuracy = []
train_loss = []
val_accuracy = []
val_loss = []
def train_epoch(model, criterion, optimizer, dataloader, epoch_num):
    epoch_loss = 0
    epoch_true = 0
    epoch_all = 0

    model.train()
    with tqdm.tqdm(enumerate(dataloader), total=len(dataloader)) as pbar:
        for i, (x, y) in pbar:
            x = x.to(device)
            y = y.to(device)
            yhat = model(x)

            loss = criterion(yhat, y)
            epoch_loss += float(loss)

            predictions = yhat.argmax(-1)
            epoch_all += len(predictions)
            epoch_true += (predictions == y).sum()
            pbar.set_description(f'[Epoch {epoch_num}]\t[{datetime.now().strftime("%H:%M:%S")}]\tTrain Loss: {epoch_loss / (i + 1)}\tTrain Accuracy: {epoch_true * 100. / epoch_all:.2f}')

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    train_accuracy.append(epoch_true / epoch_all)
    train_loss.append(epoch_loss / len(dataloader))


def eval_epoch(model, criterion, dataloader, label, epoch_num):
    epoch_loss = 0
    epoch_true = 0
    epoch_all = 0

    model.eval()
    with torch.no_grad(), tqdm.tqdm(enumerate(dataloader), total=len(dataloader)) as pbar:
        for i, (x, y) in pbar:
            x = x.to(device)
            y = y.to(device)
            yhat = model(x)

            loss = criterion(yhat, y)
            epoch_loss += float(loss)

            predictions = yhat.argmax(-1)
            epoch_all += len(predictions)
            epoch_true += (predictions == y).sum()
            pbar.set_description(f'[Epoch {epoch_num}]\t[{datetime.now().strftime("%H:%M:%S")}]\t{label} Loss: {epoch_loss / (i + 1)}\t{label} Accuracy: {epoch_true * 100. / epoch_all:.2f}')
    if label == 'Validation':
      val_accuracy.append(epoch_true / epoch_all)
      val_loss.append(epoch_loss / len(dataloader))


epochs = 164

for e in range(epochs):
    train_epoch(model, criterion, optimizer, train_loader, e + 1)
    eval_epoch(model, criterion, val_loader, 'Validation', e + 1)


[Epoch 1]	[17:29:08]	Train Loss: 0.4492764952778816	Train Accuracy: 85.86: 100%|██████████| 200/200 [01:57<00:00,  1.70it/s]
[Epoch 1]	[17:29:22]	Validation Loss: 0.2105109930038452	Validation Accuracy: 93.86: 100%|██████████| 25/25 [00:14<00:00,  1.72it/s]
[Epoch 2]	[17:31:18]	Train Loss: 0.3082373552210629	Train Accuracy: 90.23: 100%|██████████| 200/200 [01:56<00:00,  1.72it/s]
[Epoch 2]	[17:31:33]	Validation Loss: 0.17810454711318016	Validation Accuracy: 94.99: 100%|██████████| 25/25 [00:14<00:00,  1.77it/s]
[Epoch 3]	[17:33:29]	Train Loss: 0.28452169010415673	Train Accuracy: 91.20: 100%|██████████| 200/200 [01:56<00:00,  1.72it/s]
[Epoch 3]	[17:33:43]	Validation Loss: 0.17539528300985693	Validation Accuracy: 94.74: 100%|██████████| 25/25 [00:14<00:00,  1.74it/s]
[Epoch 4]	[17:35:40]	Train Loss: 0.24797960175201297	Train Accuracy: 91.98: 100%|██████████| 200/200 [01:56<00:00,  1.72it/s]
[Epoch 4]	[17:35:54]	Validation Loss: 0.175014083776623	Validation Accuracy: 94.61: 100%|████████

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [ ]:
!pip freeze > requirements.txt


Usage: cp [OPTION]... [-T] SOURCE DEST
  or:  cp [OPTION]... SOURCE... DIRECTORY
  or:  cp [OPTION]... -t DIRECTORY SOURCE...
Copy SOURCE to DEST, or multiple SOURCE(s) to DIRECTORY.

Mandatory arguments to long options are mandatory for short options too.
  -a, --archive                same as -dR --preserve=all
      --attributes-only        don't copy the file data, just the attributes
      --backup[=CONTROL]       make a backup of each existing destination file
  -b                           like --backup but does not accept an argument
      --copy-contents          copy contents of special files when recursive
  -d                           same as --no-dereference --preserve=links
  -f, --force                  if an existing destination file cannot be
                                 opened, remove it and try again (this option
                                 is ignored when the -n option is also used)
  -i, --interactive            prompt before overwrite (overrides a previo

In [ ]:
!pip install --upgrade google-colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp checkpoint.pth /content/drive/MyDrive